In [1]:
import numpy as np
import json

In [2]:
from pymongo import MongoClient

In [3]:
client = MongoClient()
db = client.cmv
posts_collection = db.posts
tl_comments_collection = db.tl_comments
deltad_replies_collection = db.deltad_replies

In [4]:
rand_state = 42

In [5]:
# establish the set of texts to use (ie posts vs comments)
doctype = 'comment'

In [6]:
# getting posts with (or without) deltas awarded
deltad_post_gen = posts_collection.find( {'tl_comment_delta_parents': {"$exists": True}})
if doctype == 'post':
    undeltad_post_gen = posts_collection.find( {'tl_comment_delta_parents': {"$exists": False}})

In [7]:
if doctype == 'comment':
    # get ids of Top Level Comments that resulted in deltas AND list of all TL Comment IDs for posts where some delta was awarded by OP
    post_comment_ids = [(post['tl_comment_delta_parents'], post['comment_ids']) for post in deltad_post_gen]
    (deltad_tl_comment_ids, all_tl_comment_ids) = zip(*post_comment_ids)

    # flatten lists of lists
    deltad_tl_comment_ids = [item for sublist in deltad_tl_comment_ids for item in sublist]
    all_tl_comment_ids = [item for sublist in all_tl_comment_ids for item in sublist]

    # get ids of TL Comments that did not result in deltas from posts where OP did award deltas
    undeltad_tl_comment_ids = list(set(all_tl_comment_ids) - set(deltad_tl_comment_ids))


In [8]:
if doctype == 'comment':
    # retrieve TL comments resulting in deltas by id
    # if I reimport: deltad_tl_comment_gen = tl_comments_collection.find( {'comment_id': {"$in": deltad_tl_comment_ids}})
    deltad_tl_comment_gen = tl_comments_collection.find( {'$and': [{'comment_id': {"$in": deltad_tl_comment_ids}},{'comment_text': {"$ne": '[deleted]'}}]})
    # retrieve TL comments NOT resulting in deltas
    undeltad_tl_comment_gen = tl_comments_collection.find( {'$and': [{'comment_id': {"$in": undeltad_tl_comment_ids}},{'comment_text': {"$ne": '[deleted]'}}]})

In [9]:
# establish the set of texts to use (ie posts vs comments)
if doctype == 'comment':
    deltad_docs = [{'id': comment[f'{doctype}_id'], 'text': comment[f'{doctype}_text'], 'label': 1} for comment in deltad_tl_comment_gen]
    undeltad_docs = [{'id': comment[f'{doctype}_id'], 'text': comment[f'{doctype}_text'], 'label': 0} for comment in undeltad_tl_comment_gen]
    
elif doctype == 'post':
    deltad_post_gen = posts_collection.find( {'tl_comment_delta_parents': {"$exists": True}})
    undeltad_post_gen = posts_collection.find( {'tl_comment_delta_parents': {"$exists": False}})
    
    deltad_docs = [{'id': post[f'{doctype}_id'], 'text': post[f'{doctype}_text'], 'label': 1} for post in deltad_post_gen]
    undeltad_docs = [{'id': post[f'{doctype}_id'], 'text': post[f'{doctype}_text'], 'label': 0} for post in undeltad_post_gen]

In [10]:
#train test split
test_split_d = int(0.2*len(deltad_docs))
test_split_u = int(0.2*len(undeltad_docs))

np.random.seed(seed=rand_state)
np.random.shuffle(deltad_docs)
np.random.shuffle(undeltad_docs)

test_docs = deltad_docs[0:test_split_d]
test_docs.extend(undeltad_docs[0:test_split_u])

train_docs= deltad_docs[test_split_d::]
train_docs.extend(undeltad_docs[test_split_u::])

In [11]:
train_tuples = [(doc['id'],doc['text'],doc['label']) for doc in train_docs]
test_tuples = [(doc['id'],doc['text'],doc['label']) for doc in test_docs]

#train_docs = []
#test_docs = []

(train_ids, train_texts, train_labels) = zip(*train_tuples)
(test_ids, test_texts, test_labels) = zip(*test_tuples)

train_tuples = []
test_tuples = []


In [12]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from sklearn.feature_extraction.text import CountVectorizer
import string

In [13]:
def clean_text(texts, tokenizer, stemmer):
    stemmer_inst = stemmer()
    tokenizer_inst = tokenizer()
    cleaned_texts = []
    for text in texts:
        #cleaned_words = []
        
        #strip punctuation and digits from whole post
        to_replace = [punc for punc in string.punctuation+string.digits if punc!="'"]
        translate_dict = {key: ' ' for key in to_replace}
        translate_dict["'"] = ''
        replacement_table = str.maketrans(translate_dict)
        stripped_text = text.translate(replacement_table)
        
        #lower case post
        lowered_text = stripped_text.lower()
        cleaned_texts.append(lowered_text)
        #tokenized_post = tokenizer_inst.tokenize(stripped_post) 
        #for word in tokenized_post:
            #low_word = stemmer_inst.stem(word.lower())
            #cleaned_words.append(low_word)
            #remove stopwords?
            #if low_word not in stopwords:
            #    cleaned_words.append(low_word)
        # cleaned_posts.append(' '.join(cleaned_words))
    return cleaned_texts

In [14]:
stemmer = PorterStemmer
tokenizer = WhitespaceTokenizer

In [50]:
cleaned_train_texts = clean_text(train_texts, tokenizer, stemmer)
cleaned_test_texts = clean_text(test_texts, tokenizer, stemmer)

In [59]:
print(train_texts[0])
print(cleaned_train_texts[0])

I am an organ donor.  I want my organs to go to people who need my organs. I do not want my donation of my organs to get involved in notion of social justice, of "deserving" and so on.  The organ I donate goes to someone who is sick, because being sick is a really shitty thing.  It's not _less shitty_ when it happens to an asshole, or even to someone who doesn't want to donate their own organs.  My gift is not contingent.

I agree with you that it should not be taken for granted, I believe that it should be wildly promoted and that we should change our systems of becoming an organ donor.  As you note, this is about saving lives.  I believe it to be bad to create a system that implicitly values one life greater than the other, and that is what your proposal does.  It says that you deserve to live because you are an organ donor and you don't because you aren't.

And...Get your shit together Canada!
i am an organ donor   i want my organs to go to people who need my organs  i do not want m

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
my_vectorizer = CountVectorizer(max_df=0.85, min_df=25,
                                max_features=1000,
                                stop_words='english')

In [54]:
vectorized_train_texts = my_vectorizer.fit_transform(cleaned_train_texts)
vectorized_test_texts = my_vectorizer.transform(cleaned_test_texts)

In [26]:
import pandas as pd
df_cv = pd.DataFrame(vectorized_train_texts.toarray(), columns=my_vectorizer.get_feature_names())

## Unsupervised Topic Extraction

In [27]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [28]:
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation

#### LSA

In [241]:
lsa_model = TruncatedSVD(n_components=100, random_state=rand_state)
lsa_model.fit(vectorized_train_texts)

TruncatedSVD(algorithm='randomized', n_components=100, n_iter=5,
       random_state=42, tol=0.0)

In [242]:
lsa_train_texts = lsa_model.transform(vectorized_train_texts)
lsa_test_texts = lsa_model.transform(vectorized_test_texts)

In [243]:
display_topics(lsa_model,my_vectorizer.get_feature_names(),20) # We have to look at the topics before hand and then add the labels afterwards


Topic  0
peoples, key, driving, life, thing, majority, ways, wanted, theyve, thousands, reality, imagine, saying, knowledge, gone, doing, th, youve, going, personal

Topic  1
peoples, books, win, general, trade, quite, gender, greater, questions, complex, ignore, policies, sound, race, hate, data, identity, popular, currently, safety

Topic  2
got, money, truly, voted, wages, mind, talking, paying, majority, creating, stop, increasing, taught, workers, effect, status, pass, commit, save, powerful

Topic  3
truly, voted, driving, commit, life, hes, hillary, pass, thing, president, car, politicians, save, entire, vs, republican, imagine, republicans, deserve, drive

Topic  4
life, games, key, gay, play, peoples, thousands, assuming, playing, plenty, congress, gone, view, players, taxes, new, future, money, theyll, workers

Topic  5
general, wont, key, meet, trade, wages, mind, money, life, got, talking, womens, share, ignore, room, making, paying, workers, field, increasing

Topic  6
bo

#### LDA

In [59]:
n_topics = 150
n_iter = 13
lda_model = LatentDirichletAllocation(n_components=n_topics,
                                max_iter=n_iter,
                                random_state=rand_state,
                               learning_method='online')
lda_train_texts = lda_model.fit_transform(vectorized_train_texts)
lda_test_texts = lda_model.transform(vectorized_test_texts)

In [60]:
display_topics(lda_model,my_vectorizer.get_feature_names(),20) # We have to look at the topics before hand and then add the labels afterwards


Topic  0
real, lives, problems, relationship, long, healthy, global, relationships, term, experiences, thousands, lack, think, way, going, levels, totally, exists, lots, working

Topic  1
general, actions, argue, experience, certain, given, claim, example, argument, does, way, second, particular, fact, choose, think, define, role, present, generally

Topic  2
police, individuals, race, single, racist, minority, equally, random, related, exists, certainly, did, capable, personally, just, saying, say, like, comes, people

Topic  3
thing, debt, national, essentially, university, honestly, lot, takes, choices, policy, systems, mind, legal, high, havent, question, goes, elected, vote, solution

Topic  4
like, great, just, im, kind, lot, guy, friends, think, really, pretty, didnt, got, stuff, sure, know, thats, follow, shit, little

Topic  5
state, issue, illegal, eu, impossible, federal, deal, consent, essentially, totally, laws, isnt, allowing, want, happen, completely, used, nation, situ

#### PyTextRank for graph-based feature extraction

In [15]:
train_docs[0]

{'id': 'da5sh31',
 'text': 'I am an organ donor.  I want my organs to go to people who need my organs. I do not want my donation of my organs to get involved in notion of social justice, of "deserving" and so on.  The organ I donate goes to someone who is sick, because being sick is a really shitty thing.  It\'s not _less shitty_ when it happens to an asshole, or even to someone who doesn\'t want to donate their own organs.  My gift is not contingent.\n\nI agree with you that it should not be taken for granted, I believe that it should be wildly promoted and that we should change our systems of becoming an organ donor.  As you note, this is about saving lives.  I believe it to be bad to create a system that implicitly values one life greater than the other, and that is what your proposal does.  It says that you deserve to live because you are an organ donor and you don\'t because you aren\'t.\n\nAnd...Get your shit together Canada!',
 'label': 1}

In [16]:
replacement_table = str.maketrans({'\n': ' ', "'": '', '-': '', '/': ''})
train_dicts = [{'id': doc['id'], 'text': doc['text'].translate(replacement_table)} for doc in train_docs]
test_dicts = [{'id': doc['id'], 'text': doc['text'].translate(replacement_table)} for doc in test_docs]

In [17]:
len(train_dicts)

12063

In [18]:
print(train_dicts[0])

{'id': 'da5sh31', 'text': 'I am an organ donor.  I want my organs to go to people who need my organs. I do not want my donation of my organs to get involved in notion of social justice, of "deserving" and so on.  The organ I donate goes to someone who is sick, because being sick is a really shitty thing.  Its not _less shitty_ when it happens to an asshole, or even to someone who doesnt want to donate their own organs.  My gift is not contingent.  I agree with you that it should not be taken for granted, I believe that it should be wildly promoted and that we should change our systems of becoming an organ donor.  As you note, this is about saving lives.  I believe it to be bad to create a system that implicitly values one life greater than the other, and that is what your proposal does.  It says that you deserve to live because you are an organ donor and you dont because you arent.  And...Get your shit together Canada!'}


In [31]:
import pytextrank
import sys

path_stage0 = 'stage0.json'
path_stage1 = 'stage1.json'
path_stage2 = 'stage2.json'
path_stage3 = 'stage3.json'

#did 0-1000

i=0
for i, doc_dict in enumerate(train_dicts[1000::]):
    if i % 50 == 0:
        print(i)
    
    with open(path_stage0, 'w') as f:
        json.dump(doc_dict, f)
    # Stage 1    
    with open(path_stage1, 'w') as f:
        for graf in pytextrank.parse_doc(pytextrank.json_iter(path_stage0)):
            f.write("%s\n" % pytextrank.pretty_print(graf._asdict()))
            # print(pytextrank.pretty_print(graf))
    # Stage 2
    graph, ranks = pytextrank.text_rank(path_stage1)
    pytextrank.render_ranks(graph, ranks)
    with open(path_stage2, 'w') as f:
        for rl in pytextrank.normalize_key_phrases(path_stage1, ranks):
            f.write("%s\n" % pytextrank.pretty_print(rl._asdict()))
            # to view output in this notebook
            # print(pytextrank.pretty_print(rl))
    # Stage 3
    kernel = pytextrank.rank_kernel(path_stage2)
    with open(path_stage3, 'w') as f:
        for s in pytextrank.top_sentences(kernel, path_stage1):
            f.write(pytextrank.pretty_print(s._asdict()))
            f.write("\n")
            # to view output in this notebook
            # print(pytextrank.pretty_print(s._asdict()))
    # Stage 4
    phrase_list = list(set([p for p in pytextrank.limit_keyphrases(path_stage2, phrase_limit=15)]))
    phrases = ", ".join(phrase_list)
    
    sent_iter = sorted(pytextrank.limit_sentences(path_stage3, word_limit=150), key=lambda x: x[1])
    s = []

    for sent_text, idx in sent_iter:
        s.append(pytextrank.make_sentence(sent_text))

    graf_text = " ".join(s)
    
    tl_comments_collection.update_one({f'{doctype}_id': {'$eq': doc_dict['id']}},{'$set': {'key_phrases': phrase_list}})
    
    #print("**excerpts:** %s\n\n**keywords:** %s" % (graf_text, phrases), '\n\n')

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750
9800
9850
9900
9950
10000
10050
10100
10150

In [42]:
key_phrase_gen = tl_comments_collection.find({'key_phrases': {"$exists": True}})

In [43]:
key_phrases_list = [comment['key_phrases'] for comment in key_phrase_gen]

In [44]:
flat_key_phrases = [item for sublist in key_phrases_list for item in sublist]
key_phrases_list = list(set(flat_key_phrases))
print(len(key_phrases_list))

46022


In [45]:
flat_key_phrases = []

In [46]:
phrase_array = np.array(key_phrases_list)
key_phrases_list = []

In [47]:
from sklearn.feature_extraction.text import CountVectorizer

In [65]:
binary_vectorizer = CountVectorizer(max_df=0.9, min_df=5,
                                    max_features=20000,
                                    vocabulary = phrase_array,
                                    binary=False)

In [66]:
textranked_train_texts = binary_vectorizer.fit_transform(cleaned_train_texts)
textranked_test_texts = binary_vectorizer.transform(cleaned_test_texts)

#### Choosing the type of topic modeling:

In [67]:
reduced_train_texts = textranked_train_texts
reduced_test_texts = textranked_test_texts

## Supervised Classification

In [74]:
# undersampling undeltad observations:

#### SVM Classifier

In [53]:
from sklearn.metrics import confusion_matrix, roc_auc_score

In [54]:
from sklearn.svm import SVC

In [55]:
svc_model = SVC(class_weight='balanced', random_state=rand_state)
svc_model.fit(reduced_train_texts, train_labels)

SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False)

In [56]:
predicted_labels = svc_model.predict(reduced_test_texts)
conf_mat = confusion_matrix(test_labels, predicted_labels)
print(conf_mat)

[[2630   14]
 [ 362    8]]


In [58]:
print('mean accuracy is {}'.format(svc_model.score(reduced_test_texts, test_labels)))

mean accuracy is 0.8752488387524884


#### XGBoost Random Forest Classifier

In [59]:
from xgboost import XGBClassifier

In [68]:
xg_model = XGBClassifier(scale_pos_weight=7)
xg_model.fit(reduced_train_texts, train_labels)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=7, seed=None,
       silent=True, subsample=1)

In [69]:
print('train set mean accuracy is {}'.format(xg_model.score(reduced_train_texts, train_labels)))


train set mean accuracy is 0.7223742021056122


/home/eleanor/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [70]:
print('mean accuracy is {}'.format(xg_model.score(reduced_test_texts, test_labels)))
predicted_labels = xg_model.predict(reduced_test_texts)
conf_mat = confusion_matrix(test_labels, predicted_labels)
print(conf_mat)

mean accuracy is 0.6718646317186463
[[1841  803]
 [ 186  184]]


/home/eleanor/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/eleanor/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [76]:
#from sklearn.cluster import DBSCAN

In [77]:
#dbscan_model = DBSCAN(eps=0.5, min_samples=5, metric=’euclidean’, metric_params=None, algorithm=’auto’, leaf_size=30, p=None, n_jobs=1)

In [84]:
from sklearn.linear_model import LogisticRegression

In [86]:
logreg = LogisticRegression(random_state=rand_state, class_weight='balanced')
logreg.fit(reduced_train_texts, train_labels)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=42,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [90]:
predicted_test_labels = logreg.predict(reduced_test_texts)

In [91]:
print('mean accuracy is {}'.format(logreg.score(reduced_test_texts, test_labels)))
conf_mat = confusion_matrix(test_labels, predicted_test_labels)
print(conf_mat)

mean accuracy is 0.6766313082610094
[[1925  795]
 [ 211  180]]
